In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, cross_val_predict

In [ ]:
df = pd.read_csv("../datasets/titanic_train.csv")
df.head(10)

## 1.1

In [ ]:
df['Sex'].value_counts()

### Ответ
Мужчины, женщины<br>
<u>577</u>, <u>314</u> 

## 1.2

In [ ]:
survived_series = df['Survived'].value_counts()
survived_series

Из описание
0 = No, 1 = Yes

In [ ]:
survival_rate = survived_series[1] / (survived_series[0] + survived_series[1])
survival_rate

In [ ]:
survival_percent = round(survival_rate*100, 2)
survival_percent

### Ответ
38,38% выживших

## 1.3

In [ ]:
pclass_series = df['Pclass'].value_counts()
pclass_series

In [ ]:
first_class_rate = pclass_series[1] / (pclass_series[1] + pclass_series[2] + pclass_series[3])
first_class_rate

In [ ]:
first_class_percent = round(first_class_rate*100, 2)
first_class_percent

### Ответ
24,24% пассажиров первого класса

## 1.4
Какого возраста были пассажиры? Посчитайте среднее и медиану возраста пассажиров. В качестве ответа приведите два числа через пробел.

In [ ]:
age_series = df["Age"]
round(age_series.mean(), 2)

In [ ]:
age_series.median()

### Ответ
29,7 лет - средний возраст пассажиров<br>
28 лет - медиана возрастов пассажиров

## 1.5
Коррелируют ли число братьев/сестер/супругов с числом родителей/детей? Посчитайте корреляцию Пирсона между признаками SibSp и Parch.

In [ ]:
correlation_df = df[['SibSp', 'Parch']]
correlation_df.head(10)

<b>Link to the doc on corr() method in pandas:</b><br>
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.corr.html<br>

<b>Link to a theory description:</b><br>
https://en.wikipedia.org/wiki/Pearson_correlation_coefficient

<b>Some description about which correlation value is significant:</b><br>
http://support.minitab.com/en-us/minitab-express/1/help-and-how-to/modeling-statistics/regression/how-to/correlation/interpret-the-results/

In [ ]:
correlation_df.corr(method='pearson')

### Ответ
Значение корреляции - 0.414838<br>
Это является индикатором средней позитивной линейной зависимости между переменными

### 1.6. 
Какое самое популярное женское имя на корабле? Извлеките из полного имени пассажира (колонка Name) его личное имя (First Name).

In [ ]:
women_df = df[df['Sex'] == "female"]
women_df.head()

#### Правило
Если Miss, то брать то, что после Miss<br>
Если Mrs., то брать первое, что в скобках<br>
Если Mme., то брать то, что после Mme

In [ ]:
def get_first_name(name_str):
    name_list = name_str.split(" ")
    
    if "Mme." in name_list:
        # If there's a "Mme." in the name
#         print("Mme: " + str(name_list))
        return name_list[name_list.index("Mme.") + 1]
    
    if "Mrs." in name_list:
        # If there's a "Mrs." in the name
#         print("Mrs: " + str(name_str))
        if name_str.find("(") != -1:
            name = name_str[name_str.index("(")+1:]
            if name.find(" ") != -1:
                name = name[:name.index(" ")]
            else:
                name = name[:name.index(")")]
            return name
        else:
            return name_list[name_list.index("Mrs.") + 1]
    
    if "Miss." in name_list:
        # If there's a "Miss." in the name
#         print("Miss: " + str(name_list))
        return name_list[name_list.index("Miss.") + 1]

names_list = women_df["Name"].apply(get_first_name).tolist()
names_list[:10] # show 10 elements

In [ ]:
Counter(names_list).most_common(5)

In [ ]:
top_ten = Counter(names_list).most_common(5)
top_ten_names = [i[0] for i in top_ten]
top_ten_values = [i[1] for i in top_ten]

plt.xticks(range(5), top_ten_names, rotation='vertical')
plt.scatter(range(5), top_ten_values)
for i in range(5):
    plt.annotate(top_ten_values[i], xy=(i,top_ten_values[i]))
plt.show()

### Ответ
Anna - если не учитывать все вариации имени Mary

### 2. 
Оставьте в выборке четыре признака: класс пассажира (Pclass), цену билета (Fare), возраст пассажира (Age) и его пол (Sex).

In [ ]:
df.dropna(inplace=True, subset=['Age'])

df2 = df[['Pclass', 'Fare', 'Age', 'Sex']].copy()
df2.head()

### 2.1
Обратите внимание, что признак Sex имеет строковые значения

In [ ]:
df2['Sex'] = df2['Sex'].replace('male', 1)
df2['Sex'] = df2['Sex'].replace('female', 0)
df2.head(10)

### 2.2.
Выделите целевую переменную — она записана в столбце Survived.

In [ ]:
target_variable = df['Survived']
target_variable.head(10)

### 2.3
В данных есть пропущенные значения — например, для некоторых пассажиров неизвестен их возраст. Такие записи при чтении их в pandas принимают значение nan. Найдите все объекты, у которых есть пропущенные признаки, и удалите их из выборки.

In [ ]:
df2.dropna(inplace=True)
df2.head(10)

### 2.4. 
Обучите решающее дерево с параметром random_state=241 и остальными параметрами по умолчанию (речь идет о параметрах конструктора DecisionTreeСlassifier).

<b>Link to "Understanding the decision tree structure":</b><br>
http://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html<br>
<b>"Desition tree" in Scikit Learn docs</b><br>
http://scikit-learn.org/stable/modules/tree.html

<b>В данном месте обучаем просто на всех данных, без разделения на тренировочную и тестовую выборку.<br>
Это будет сделано в пункте 3.</b>

In [ ]:
estimator = DecisionTreeClassifier(random_state=241)
estimator.fit(df2, target_variable)

In [ ]:
accuracy_score(target_variable, estimator.predict(df2))

98% - высокий результат, но так как мы не делили выборку на тренировочную и тестовую, то, конечно, становится понятно, почему мы получили такой высокий результат.

### 2.5. 
Вычислите важности признаков и найдите два признака с наибольшей важностью.

Это задача называется <b>feature selection</b>.<br>
Документация на функцию SelectKBest:
http://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection

In [ ]:
print("Initial df2 shape: " + str(df2.shape))
print("df2: " + str(df2.head(10)))
SelectKBest(k=2).fit(df2, target_variable)
idxs_selected = selector.get_support(indices=True)
print("New df2 shape: " + str(df2_new.shape))
df2_new

In [ ]:
print("Initial df2 shape: " + str(df2.shape))
print("df2: " + str(df2.head(10)))

# Create and fit selector
selector = SelectKBest(k=2)
selector.fit(df2, target_variable)

# Get ids of columns to keep
ids_selected = selector.get_support(indices=True)

# Create new dataframe with only desired columns, or overwrite existing
df2_reduced = df2.iloc[:,ids_selected]

print("New df2 shape: " + str(df2_reduced.shape))
df2_reduced.head(10)

In [ ]:
df2_reduced.columns

### Ответ
<b>'Pclass' и 'Sex'</b> - два самых важных признака с помощью метода SelectKBest и одномерного статистического теста k-classif

### 3

### 3.1
Учитывая результаты решения предыдущих примеров, построить модель, прогнозирующую, спасется человек на Титанике или нет.

Сначала разделим наш сокращенный датафрейм (df2_reduced) на тестовую и тренировочную выборку в размере 70-30% (числа из головы) и поставим random_state = 241 для воспроизводимости результатов

In [ ]:
my_rand_state = 241

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df2_reduced, target_variable, 
                                                    test_size=0.3, random_state=my_rand_state)

Посмотрим, как DecisionTreeClassifier с random_state=241 справится с данными с уменьшенной размерностью и с использованием тестовых данных, которые не участвовали в обучении 

In [ ]:
estimator = DecisionTreeClassifier(random_state=my_rand_state)
estimator.fit(X_train, y_train)

In [ ]:
accuracy_score(y_test, estimator.predict(X_test))

77,2% - не очень хороший результат

### 3.2
Модель построить для двух алгоритмов, «метода ближайших соседей» и «логистической регрессии»

Scikit-learn KNearestNeighbors:<br>
http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html<br>
and KNearestNeighbors in details:<br>
http://scikit-learn.org/stable/modules/neighbors.html<br>
Scikit-learn LogisticRegression:<br>
http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

<b>KNearestNeighbours</b>
с числом соседей = 5

In [ ]:
# Educate
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train, y_train)

# Predict
y_pred = neigh.predict(X_test)

# Print report
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
accuracy_score(y_test, y_pred)

<b>LogisticRegression</b> с дефолтными параметрами (L2 как penalty, liblinear как solver, так как его советуют для маленьких датасетов)

In [ ]:
# Educate
regr = LogisticRegression()
regr.fit(X_train, y_train)

# Predict
y_pred = regr.predict(X_test)

# Print report
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
accuracy_score(y_test, y_pred)

### 3.3
Проверку модели произвести с помощью скользящего контроля.<br>


In [ ]:
scores_neigh = cross_val_score(neigh, df2_reduced, target_variable, cv=5)
print("Cross-validated scores for each step: \n{}".format(scores_neigh))

In [ ]:
scores_regr = cross_val_score(regr, df2_reduced, target_variable, cv=5)
print("Cross-validated scores for each step: \n{}".format(scores_regr))

### 3.4
Результат – сравнить точность работы модели по двум алгоритмам, в процентах с округлением до сотых.

<b>Точность метода ближайших соседей</b>

In [ ]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_neigh.mean(), scores_neigh.std() * 2))

<b>Точность метода логистической регрессии</b>

In [ ]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_regr.mean(), scores_regr.std() * 2))

### Ответ
<b>Метод ближайших соседей с числом соседей = 5</b> обладает на 2% лучшей точностью при кросс-валидации с 5 делениями датасета